In [1]:
!pip install mygene statannotations scrublet scanpy scvelo decoupler matplotlib_venn goatools gseapy scperturb biomart PyComplexHeatmap statsmodels omnipath git+https://github.com/saezlab/pypath.git --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyopenssl 23.0.0 requires cryptography<40,>=38.0.0, but you have cryptography 41.0.7 which is incompatible.


In [2]:
import subprocess
import os
import sys
import matplotlib.backends.backend_pdf
import scanpy as sc
import matplotlib.pyplot as pl
import anndata as ad
import pandas as pd
import numpy as np
import seaborn as sns
import scvelo as scv
scv.settings.verbosity=1

from pathlib import Path

# Jupyter stuff
from tqdm.notebook import tqdm
from IPython.display import clear_output
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline

# Custom functions
sys.path.insert(1, '../')
from utils import *

# scperturb package
sys.path.insert(1, '../package/src/')
from scperturb import *

from pathlib import Path
figure_path = Path('../figures/')

In [3]:
TEMPDIR = Path('/scratch/peidli/scPerturb/')

In [17]:
sorted([file.name for file in (TEMPDIR / 'LiangWang2023/').glob('*')])

['OSCAR_DM_expression_matrix.csv',
 'OSCAR_EM_expression_matrix.csv',
 'OSCAR_metadata.csv']

In [18]:
tab_DM = pd.read_csv(TEMPDIR / f'LiangWang2023/OSCAR_DM_expression_matrix.csv', index_col=0)
tab_EM = pd.read_csv(TEMPDIR / f'LiangWang2023/OSCAR_EM_expression_matrix.csv', index_col=0)
meta_df = pd.read_csv(TEMPDIR / f'LiangWang2023/OSCAR_metadata.csv', index_col=0)

In [20]:
from scipy.sparse import csr_matrix

In [30]:
tab = pd.concat([tab_DM, tab_EM], axis=1)

In [45]:
adata = sc.AnnData(csr_matrix(tab.T.values))
adata.obs_names = tab.columns
adata.var_names = tab.index
adata.obs = meta_df.loc[adata.obs_names]

In [46]:
adata.obs

,cell,barcode,sgrna,gene,sample,type
DM2_1_AAACCTGAGAGCTGCA-1,AAACCTGAGAGCTGCA-1,Spib-3,CTGGGTCGAGGGTCCCCATG,Spib,DM2_1,DM
DM2_1_AAACCTGCAAGTACCT-1,AAACCTGCAAGTACCT-1,Spic-3,CCCTTCTTCCAGCAAAAGGG,Spic,DM2_1,DM
DM2_1_AAACCTGCAAGTTCTG-1,AAACCTGCAAGTTCTG-1,Onecut1-3,CCACTATGCTCATCCCGGCG,Onecut1,DM2_1,DM
DM2_1_AAACCTGCAGATTGCT-1,AAACCTGCAGATTGCT-1,non_target-1,AAGGCGTAAACGAGTACACG,Non-Targeting,DM2_1,DM
DM2_1_AAACCTGCATGATCCA-1,AAACCTGCATGATCCA-1,Lyl1-1,AGGGTGGTAATGTACAGCCA,Lyl1,DM2_1,DM
...,...,...,...,...,...,...
EM2_3_TTTGTCACACTTGGAT-1,TTTGTCACACTTGGAT-1,Peg3-2,GAAGGGAGCGCATTTAGGGG,Peg3,EM2_3,EM
EM2_3_TTTGTCACAGGTCCAC-1,TTTGTCACAGGTCCAC-1,Nfe2-2,GAAGACCCAGAATCTGACTC,Nfe2,EM2_3,EM
EM2_3_TTTGTCAGTACCGCTG-1,TTTGTCAGTACCGCTG-1,Mxd3-2,CTGGCGCGCTGAACAGTGGG,Mxd3,EM2_3,EM
EM2_3_TTTGTCAGTCTGGTCG-1,TTTGTCAGTCTGGTCG-1,H1fx-3,GGAGACCATCCGCAAGCTGG,H1fx,EM2_3,EM


In [47]:
# Harmonize Metadata
adata.obs_names = [x.split('-')[0] for x in adata.obs_names]
adata.var.index.name='gene_symbol'
adata.obs.drop(['cell'], axis=1, inplace=True)
adata.obs.index.name = 'cell_barcode'
adata.obs.rename({
    'barcode': 'guide_id',
    'sgrna': 'guide_sequence',
    'gene': 'perturbation',
    'type': 'medium'
}, axis=1, inplace=True)
adata.obs.perturbation[adata.obs.perturbation=='Non-Targeting'] = 'control'
adata.obs.medium.replace({'DM': 'Differentiation Medium', 'EM': 'Expansion Medium'}, inplace=True)
adata.obs['perturbation_type'] = 'CRISPR-cas9'
adata.obs['nperts']= 1 - adata.obs.perturbation.str.count('control')
adata.obs['organism'] = 'mouse'
adata.obs['tissue_type'] = 'organoid'
adata.obs['disease'] = 'healthy'
adata.obs['celltype'] = 'hepatocyte'
adata.obs['cancer'] = False
adata.obs['cell_line'] = 'intrahepatic cholangiocyte organoids'

In [48]:
adata.obs

,guide_id,guide_sequence,perturbation,sample,medium,perturbation_type,nperts,organism,tissue_type,disease,celltype
cell_barcode,,,,,,,,,,,
DM2_1_AAACCTGAGAGCTGCA,Spib-3,CTGGGTCGAGGGTCCCCATG,Spib,DM2_1,Differentiation Medium,CRISPR-cas9,1,mouse,organoid,healthy,hepatocyte
DM2_1_AAACCTGCAAGTACCT,Spic-3,CCCTTCTTCCAGCAAAAGGG,Spic,DM2_1,Differentiation Medium,CRISPR-cas9,1,mouse,organoid,healthy,hepatocyte
DM2_1_AAACCTGCAAGTTCTG,Onecut1-3,CCACTATGCTCATCCCGGCG,Onecut1,DM2_1,Differentiation Medium,CRISPR-cas9,1,mouse,organoid,healthy,hepatocyte
DM2_1_AAACCTGCAGATTGCT,non_target-1,AAGGCGTAAACGAGTACACG,control,DM2_1,Differentiation Medium,CRISPR-cas9,0,mouse,organoid,healthy,hepatocyte
DM2_1_AAACCTGCATGATCCA,Lyl1-1,AGGGTGGTAATGTACAGCCA,Lyl1,DM2_1,Differentiation Medium,CRISPR-cas9,1,mouse,organoid,healthy,hepatocyte
...,...,...,...,...,...,...,...,...,...,...,...
EM2_3_TTTGTCACACTTGGAT,Peg3-2,GAAGGGAGCGCATTTAGGGG,Peg3,EM2_3,Expansion Medium,CRISPR-cas9,1,mouse,organoid,healthy,hepatocyte
EM2_3_TTTGTCACAGGTCCAC,Nfe2-2,GAAGACCCAGAATCTGACTC,Nfe2,EM2_3,Expansion Medium,CRISPR-cas9,1,mouse,organoid,healthy,hepatocyte
EM2_3_TTTGTCAGTACCGCTG,Mxd3-2,CTGGCGCGCTGAACAGTGGG,Mxd3,EM2_3,Expansion Medium,CRISPR-cas9,1,mouse,organoid,healthy,hepatocyte


In [41]:
adata.var

""
gene_symbol
Xkr4
Rp1
Sox17
Mrpl15
Lypla1
...
AC125149.3
AC168977.1
AC149090.1
